In [1]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda


In [21]:
class GI_NN(nn.Module):
    def __init__(self, output_channels):
        super(GI_NN, self).__init__()
        self.gnn = nn.GRU(512, 128, 20)
        self.first_layer = nn.Conv1d(6, 1, 1)
        self.fc = nn.Linear(128, 64)
        self.drop_out = nn.Dropout(0.25)
        self.last_layer = nn.Linear(64, output_channels)

    def forward(self, x):
        x_list = []
        for i in range(len(x)):
            x_list.append(self.first_layer(x[i]))
        a = torch.cat(tuple(x_list), 1)
        a = torch.unsqueeze(torch.sum(a, 1), 0)
        b = self.gnn(a)
        c = self.fc(b[0])
        d = self.drop_out(c)
        z = self.last_layer(d)

        return z

In [22]:
model = GI_NN(output_channels=2)
model.to(DEVICE)

GI_NN(
  (gnn): GRU(512, 128, num_layers=20)
  (first_layer): Conv1d(6, 1, kernel_size=(1,), stride=(1,))
  (fc): Linear(in_features=128, out_features=64, bias=True)
  (drop_out): Dropout(p=0.25, inplace=False)
  (last_layer): Linear(in_features=64, out_features=2, bias=True)
)

In [23]:
x_list = []
for _ in range(6):
    x_list.append(torch.randn(1, 6, 512).to(DEVICE))
out = model(x_list)

In [25]:
print(out)

tensor([[[-0.0975,  0.0107]]], device='cuda:0', grad_fn=<AddBackward0>)


GI_NN(
  (gnn): GRU(512, 128, num_layers=20)
  (first_layer): Conv1d(6, 1, kernel_size=(1,), stride=(1,))
  (fc): Linear(in_features=128, out_features=64, bias=True)
  (drop_out): Dropout(p=0.25, inplace=False)
  (last_layer): Linear(in_features=64, out_features=2, bias=True)
)
